In [7]:
# ==============================================================================
# 1. SETUP - INSTALL AND IMPORT LIBS
# ==============================================================================
!pip install langgraph langchain_core==0.1.52 --quiet

from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.sqlite import SqliteSaver
from typing import TypedDict, Annotated, List
import operator

# ==============================================================================
# 2. DEFINE AND BUILD THE VERIFICATION SUBGRAPH
# ==============================================================================
class VerificationState(TypedDict):
    candidate_name: str
    verification_steps: Annotated[List[str], operator.add]

def verify_experience(state: VerificationState) -> dict:
    print("  (Subgraph: Verifying candidate's experience...)")
    return {"verification_steps": ["Experience Verified via background check."]}

def verify_skills(state: VerificationState) -> dict:
    print("  (Subgraph: Verifying candidate's technical skills...)")
    return {"verification_steps": ["Skills Verified via technical assessment."]}

def create_verification_subgraph():
    workflow = StateGraph(VerificationState)
    workflow.add_node("experience", verify_experience)
    workflow.add_node("skills", verify_skills)
    workflow.add_edge(START, "experience")
    workflow.add_edge("experience", "skills")
    workflow.add_edge("skills", END)
    return workflow.compile()

# ==============================================================================
# 3. DEFINE AND BUILD THE MAIN HIRING WORKFLOW
# ==============================================================================
class MainState(TypedDict):
    candidate_name: str
    hr_approval: str
    manager_approval: str
    verification_report: dict
    status: str

def initial_screening(state: MainState) -> dict:
    print("Phase 1: Initial automated screening is complete.")
    return {}

def run_verification_subgraph(state: MainState) -> dict:
    print("\nPhase 2: Entering Verification Subgraph...")
    subgraph = create_verification_subgraph()
    sub_input = {"candidate_name": state["candidate_name"], "verification_steps": []}
    report = subgraph.invoke(sub_input)
    print("...Exiting Verification Subgraph.")
    return {"verification_report": report}

def final_decision(state: MainState) -> dict:
    print("\nPhase 3: Making Final Decision...")
    if state.get("manager_approval") == "approve":
        return {"status": "HIRED"}
    else:
        return {"status": "REJECTED"}

# ==============================================================================
# 4. RUN THE INTEGRATED SYSTEM
# ==============================================================================
if __name__ == "__main__":
    workflow = StateGraph(MainState)
    workflow.add_node("screening", initial_screening)
    workflow.add_node("hr_approval_node", lambda x: x)  # Placeholder for interrupt
    workflow.add_node("verification", run_verification_subgraph)
    workflow.add_node("manager_approval_node", lambda x: x)  # Placeholder for interrupt
    workflow.add_node("decision", final_decision)

    workflow.add_edge(START, "screening")
    workflow.add_edge("screening", "hr_approval_node")
    workflow.add_edge("hr_approval_node", "verification")
    workflow.add_edge("verification", "manager_approval_node")
    workflow.add_edge("manager_approval_node", "decision")
    workflow.add_edge("decision", END)

    memory = SqliteSaver.from_conn_string(":memory:")
    app = workflow.compile(
        checkpointer=memory,
        interrupt_before=["hr_approval_node", "manager_approval_node"]
    )

    # ✅ Proper checkpoint config
    thread_config = {"configurable": {"thread_id": "integrated-hire-007"}}

    print("--- Running until first interruption (HR Approval) ---")
    app.invoke({"candidate_name": "Alice"}, config=thread_config)
    print("--- WORKFLOW PAUSED ---")

    print("\n--- Resuming with HR Approval ---")
    app.update_state(thread_config, {"hr_approval": "approve"})   # update state
    app.invoke(None, config=thread_config)  # resume
    print("--- WORKFLOW PAUSED ---")

    print("\n--- Resuming with Manager Approval ---")
    app.update_state(thread_config, {"manager_approval": "approve"})  # update state
    final_state = app.invoke(None, config=thread_config)  # resume to end

    print("\n" + "="*50)
    print("--- Final System State ---")
    print(final_state)
    print(f"Final hiring status for {final_state['candidate_name']}: {final_state['status']}")


--- Running until first interruption (HR Approval) ---
Phase 1: Initial automated screening is complete.
--- WORKFLOW PAUSED ---

--- Resuming with HR Approval ---

Phase 2: Entering Verification Subgraph...
  (Subgraph: Verifying candidate's experience...)
  (Subgraph: Verifying candidate's technical skills...)
...Exiting Verification Subgraph.
--- WORKFLOW PAUSED ---

--- Resuming with Manager Approval ---

Phase 3: Making Final Decision...

--- Final System State ---
{'candidate_name': 'Alice', 'hr_approval': 'approve', 'manager_approval': 'approve', 'verification_report': {'candidate_name': 'Alice', 'verification_steps': ['Experience Verified via background check.', 'Skills Verified via technical assessment.']}, 'status': 'HIRED'}
Final hiring status for Alice: HIRED
